In [14]:
# Use Google Collab
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()


In [13]:
# ✅ Works in Google Colab – interactive K-Means playground (with sample data)
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, FloatText, HBox, VBox, Label, Output

out = Output()

# --------------------------------------------------------------------
# Initial setup
# --------------------------------------------------------------------
points = []
centroids = []
assignments = []

def plot_state(iteration=None):
    with out:
        out.clear_output(wait=True)
        plt.figure(figsize=(6,6))
        plt.xlim(0,10)
        plt.ylim(0,10)
        plt.title(f"K-Means Playground{'' if iteration is None else f'  (Iteration {iteration})'}")

        if len(points):
            pts = np.array(points)
            if len(assignments):
                for k in range(len(centroids)):
                    cluster_pts = pts[np.array(assignments)==k]
                    plt.scatter(cluster_pts[:,0], cluster_pts[:,1], s=60, label=f'Cluster {k+1}')
            else:
                plt.scatter(pts[:,0], pts[:,1], s=60, color='blue', label='Points')
        if len(centroids):
            cents = np.array(centroids)
            plt.scatter(cents[:,0], cents[:,1], s=180, color='red', marker='X', edgecolor='black', label='Centroids')

        plt.legend()
        plt.show()

# --------------------------------------------------------------------
# K-Means logic
# --------------------------------------------------------------------
def assign_clusters():
    global assignments
    if not len(points) or not len(centroids): return
    pts, cents = np.array(points), np.array(centroids)
    distances = np.linalg.norm(pts[:,None,:]-cents[None,:,:], axis=2)
    assignments = np.argmin(distances, axis=1)

def update_centroids():
    global centroids
    if not len(assignments): return
    newc = []
    for k in range(len(centroids)):
        cluster_pts = [points[i] for i,a in enumerate(assignments) if a==k]
        newc.append(np.mean(cluster_pts, axis=0) if cluster_pts else centroids[k])
    centroids[:] = newc

# --------------------------------------------------------------------
# UI setup
# --------------------------------------------------------------------
def make_point_box(i, x, y):
    return [FloatText(value=x, description=f"x{i+1}"), FloatText(value=y, description=f"y{i+1}")]

def make_centroid_box(i, x, y):
    return [FloatText(value=x, description=f"cx{i+1}"), FloatText(value=y, description=f"cy{i+1}")]

def on_apply(b):
    global points, centroids, assignments
    points = [[pb[0].value, pb[1].value] for pb in point_boxes if not np.isnan(pb[0].value)]
    centroids = [[cb[0].value, cb[1].value] for cb in centroid_boxes if not np.isnan(cb[0].value)]
    assignments.clear()
    plot_state()

def on_next(b):
    assign_clusters()
    update_centroids()
    plot_state()
    try:
        n = int(iteration_btn.description.split()[-1]) + 1
    except:
        n = 2
    iteration_btn.description = f"Next Iteration {n}"

def on_reset(b):
    global points, centroids, assignments
    points.clear(); centroids.clear(); assignments.clear()
    for pb in point_boxes: pb[0].value, pb[1].value = 0, 0
    for cb in centroid_boxes: cb[0].value, cb[1].value = 0, 0
    iteration_btn.description = "Next Iteration 1"
    plot_state()

# --------------------------------------------------------------------
# Example data for practice
# --------------------------------------------------------------------
sample_points = [
    (1, 1.5), (1.2, 1.8), (2, 2), (2.2, 1.3), (2.5, 2.5),
    (6, 6), (6.2, 6.5), (7, 6.8), (6.8, 6), (7.5, 7),
    (3.5, 8)
]
sample_centroids = [(2, 1.5), (6.5, 6.5), (3, 7)]

n_points, n_centroids = len(sample_points), len(sample_centroids)
point_boxes = [make_point_box(i, *p) for i,p in enumerate(sample_points)]
centroid_boxes = [make_centroid_box(i, *c) for i,c in enumerate(sample_centroids)]

apply_btn = Button(description="Apply Positions", button_style='info')
iteration_btn = Button(description="Next Iteration 1", button_style='success')
reset_btn = Button(description="Reset", button_style='danger')

apply_btn.on_click(on_apply)
iteration_btn.on_click(on_next)
reset_btn.on_click(on_reset)

control_grid = VBox(
    [Label("Points (x,y):")] +
    [HBox(pb) for pb in point_boxes] +
    [Label("Centroids (x,y):")] +
    [HBox(cb) for cb in centroid_boxes] +
    [HBox([apply_btn, iteration_btn, reset_btn])]
)

display(VBox([control_grid, out]))
plot_state()
